# Basic Introduction to TensorFlow

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

import sys
import input_data
import tensorflow as tf

tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Tensors
3 # a rank 0 tensor; this is a scalar with shape []
[1. ,2., 3.] # a rank 1 tensor; this is a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

3

[1.0, 2.0, 3.0]

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

In [3]:
#A computational graph is a series of TensorFlow operations arranged into a graph of nodes. 

#TensorFlow Core programs consist of two discrete sections:
#1.	Building the computational graph.
#2.	Running the computational graph.

# Constants

In [3]:
node1 = tf.compat.v1.constant(3.0, dtype=tf.compat.v1.float32)
node2 = tf.compat.v1.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [4]:
#Notice that printing the nodes does not output the values 3.0 and 4.0 as you might expect. 
#Instead, they are nodes that, when evaluated, would produce 3.0 and 4.0, respectively. 
#To actually evaluate the nodes, we must run the computational graph within a session. 

sess = tf.compat.v1.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [5]:
#more complicated computations
node3 = tf.compat.v1.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


# Placeholders

In [6]:
#A graph can be parameterized to accept external inputs, known as placeholders.
a = tf.compat.v1.placeholder(tf.float32)
b = tf.compat.v1.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [7]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[3. 7.]


In [8]:
#more complex computations
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

22.5


# Variables

In [10]:
#In ML we typically want a model that can take arbitrary inputs.
#To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. 
#Variables allow us to add trainable parameters to a graph. 

In [9]:
W = tf.compat.v1.Variable([.3], dtype=tf.float32)
b = tf.compat.v1.Variable([-.3], dtype=tf.float32)
x = tf.compat.v1.placeholder(tf.float32)
linear_model = W * x + b


In [12]:
#Constants are initialized when you call tf.constant, and their value can never change. 
#By contrast, variables are not initialized when you call tf.Variable. To initialize 
#all the variables in a TensorFlow program, you must explicitly call a special 
#operation as follows

In [10]:
init=tf.compat.v1.global_variables_initializer()
sess.run(init)
sess.run([W,b])

[array([0.3], dtype=float32), array([-0.3], dtype=float32)]

In [11]:
#Since x is a placeholder, we can evaluate linear_model 
#for several values of x simultaneously as follows
print(sess.run(linear_model, {x:[1,2,3,4]}))

[0.         0.3        0.6        0.90000004]


# How accurate is the model?

In [15]:
#We created a model. How good it is?
#To evaluate the model on training data, we need a y placeholder to provide the desired values, 
#and we need to write a loss function.

In [12]:
y = tf.compat.v1.placeholder(tf.float32)
squared_deltas = tf.compat.v1.square(linear_model - y)
loss = tf.compat.v1.reduce_sum(squared_deltas)
#tf.reduce_sum sums all the squared errors to create a single scalar 
sess.run([loss,b,W,linear_model,squared_deltas,y], {x:[1,2,3,4], y:[0,-1,-2,-3]})

[23.66,
 array([-0.3], dtype=float32),
 array([0.3], dtype=float32),
 array([0.        , 0.3       , 0.6       , 0.90000004], dtype=float32),
 array([ 0.       ,  1.6899998,  6.7599993, 15.210001 ], dtype=float32),
 array([ 0., -1., -2., -3.], dtype=float32)]

In [14]:
#let us improve the model manually
fixW = tf.compat.v1.assign(W, [-1.])
fixb = tf.compat.v1.assign(b, [1.])
print(sess.run([b,W]))
sess.run([fixW, fixb])
print(sess.run([b,W]))
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}), sess.run(b), sess.run(W))

[array([-0.3], dtype=float32), array([0.3], dtype=float32)]


[array([-1.], dtype=float32), array([1.], dtype=float32)]

[array([1.], dtype=float32), array([-1.], dtype=float32)]
0.0 [1.] [-1.]


In [18]:
#Yay! we rightly guessed the values of W and b

# Learning our first TensorFlow model

In [15]:
%%time
optimizer = tf.compat.v1.train.GradientDescentOptimizer(0.02)
train = optimizer.minimize(loss)
sess.run(init) # reset values to incorrect defaults.
print(sess.run([W, b]))
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})
  if i%100==0:
    print("Iter", i, ":",sess.run([W, b]))
print(sess.run([W, b]))

[array([0.3], dtype=float32), array([-0.3], dtype=float32)]
Iter 0 : [array([-0.73999995], dtype=float32), array([-0.612], dtype=float32)]
Iter 100 : [array([-0.9541008], dtype=float32), array([0.8650506], dtype=float32)]
Iter 200 : [array([-0.995936], dtype=float32), array([0.98805124], dtype=float32)]
Iter 300 : [array([-0.99964017], dtype=float32), array([0.99894214], dtype=float32)]
Iter 400 : [array([-0.9999681], dtype=float32), array([0.99990636], dtype=float32)]
Iter 500 : [array([-0.99999714], dtype=float32), array([0.9999916], dtype=float32)]
Iter 600 : [array([-0.9999995], dtype=float32), array([0.9999987], dtype=float32)]
Iter 700 : [array([-0.9999995], dtype=float32), array([0.9999987], dtype=float32)]
Iter 800 : [array([-0.9999995], dtype=float32), array([0.9999987], dtype=float32)]
Iter 900 : [array([-0.9999995], dtype=float32), array([0.9999987], dtype=float32)]
[array([-0.9999995], dtype=float32), array([0.9999987], dtype=float32)]
CPU times: user 781 ms, sys: 105 ms, t

# Complete Program -- Linear Regression Model

In [16]:
import tensorflow as tf

# Model parameters
#W = tf.Variable([.3], dtype=tf.float32)

W = tf.compat.v1.Variable([[.3, -0.4]], dtype=tf.compat.v1.float32)
b = tf.compat.v1.Variable([-.3], dtype=tf.compat.v1.float32)

# Model input and output

x = tf.compat.v1.placeholder(tf.compat.v1.float32)
y = tf.compat.v1.placeholder(tf.compat.v1.float32)

linear_model = tf.compat.v1.matmul(W,tf.compat.v1.transpose(x)) + b

#linear_model =W*x + b
# loss

loss = tf.compat.v1.reduce_sum(tf.compat.v1.square(linear_model - y)) # sum of the squares

# optimizer
optimizer = tf.compat.v1.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# training data
#x_train = [1,2,3,4]
x_train = [[1,2],[2,3],[3,4],[4,5]]
y_train = [0,-1,-2,-3]
# training loop
init = tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session()
sess.run(init) 
for i in range(1000):
  sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [[-0.866666   -0.13333294]] b: [1.13333] loss: 5.2580162e-12


# tf.contrib.learn

Tensorflow exposes low level (tf.train) as well as high level APIs (tf.contrib)

tf.contrib.learn is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:
 - running training loops
 - running evaluation loops
 - managing data sets
 - managing feeding


# Linear Regression Model using tf.contrib.learn

In [9]:
import tensorflow as tf
import numpy as np

# Declare list of features. We only have one real-valued feature.
features = [tf.compat.v1 contrib.layers.real_valued_column("x", dimension=1)]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)
#optimizer = tf.train.GradientDescentOptimizer(0.01)

# We use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_train}, y_train, batch_size=4, num_epochs=1000)
eval_input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_eval}, y_eval, batch_size=4, num_epochs=1000)

# We can invoke 500 training steps by invoking the method and passing the
# training data set.
# num_epochs and steps are redundant parameters. Bad design in TF! Uses "steps" if steps<num_epochs
estimator.fit(input_fn=input_fn, steps=500)

# Here we evaluate how well our model did.
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [22]:
import tensorflow as tf
import numpy as np
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_train}, y_train, batch_size=4, num_epochs=1000)
estimator.fit(input_fn=input_fn, steps=500)
train_loss = estimator.evaluate(input_fn=input_fn)
print("train loss: %r"% train_loss)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe460594ba8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmp1vltzqbp'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp1vltzqbp/mo

LinearRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x7fe46175a4e0>, 'feature_columns': [_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)], 'optimizer': None, 'gradient_clip_norm': None, 'joint_weights': False})

INFO:tensorflow:Starting evaluation at 2019-07-26T03:50:40Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1vltzqbp/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-26-03:50:41
INFO:tensorflow:Saving dict for global step 500: global_step = 500, loss = 0.0008869252
train loss: {'loss': 0.0008869252, 'global_step': 500}


# Custom (LinearRegressor) Model using tf.contrib.learn

In [23]:
#To define a custom model we simply provide Estimator a 
#function model_fn that tells tf.contrib.learn how it can 
#evaluate predictions, training steps, and loss.

In [24]:
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
#mode can be train, evaluate or infer
def model(features, labels, mode):
  # Build a linear model and predict values
  print(mode)
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the appropriate functionality.
  return tf.contrib.learn.ModelFnOps(mode=mode, predictions=y, loss=loss, train_op=train)

#estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)
estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, 4, num_epochs=1000)
eval_input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_eval}, y_eval, batch_size=4, num_epochs=1000)
predict_input_fn= tf.contrib.learn.io.numpy_input_fn({"x":x_eval}, batch_size=4, num_epochs=1)
# train
estimator.fit(input_fn=input_fn, steps=1000) #passes mode as train
# Here we evaluate how well our model did. 
train_loss = estimator.evaluate(input_fn=input_fn) #passes mode as evaluate
eval_loss = estimator.evaluate(input_fn=eval_input_fn)#passes mode as evaluate
list(estimator.predict(input_fn=predict_input_fn))#passes mode as infer
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe458565780>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpfzx3rbvk'}
train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpfzx3r

Estimator(params=None)

eval
INFO:tensorflow:Starting evaluation at 2019-07-26T03:50:43Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpfzx3rbvk/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-26-03:50:44
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 1.8356403e-11
eval
INFO:tensorflow:Starting evaluation at 2019-07-26T03:50:44Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpfzx3rbvk/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-26-03:50:45
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101007465
infer
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpfzx3rbvk/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done run

[-3.999996165599135,
 -6.999990729199203,
 -1.000001601999067,
 -3.4141323775882526e-06]

train loss: {'loss': 1.8356403e-11, 'global_step': 1000}
eval loss: {'loss': 0.0101007465, 'global_step': 1000}
